# Final Project
### Import precomp libs


In [1]:
#import the spacy english model
# "python -m spacy download en"   to download english models
import spacy
from spacy import attrs
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
import pickle

In [2]:
nlp = spacy.load('en') # this should take some time like 10s to load
# %load_ext line_profiler

In [14]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv').fillna('')
test = pd.read_csv('../data/test.csv').fillna('')
print train.shape, test.shape

(404290, 6) (2345796, 3)


In [18]:
print(type(train.loc))

<class 'pandas.core.indexing._LocIndexer'>


In [12]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

parse("")

Counter()

In [10]:
sentence = 'What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts'.decode('utf-8')
p = nlp(sentence)
np_array = p.to_array([attrs.LOWER, attrs.POS, attrs.ENT_TYPE, attrs.IS_ALPHA])

In [ ]:
doc = 

In [ ]:
train['nlp1'] = ''
train.head(20)
for i in range(len(train)):
    if i % 10000 == 0: print i
    train['nlp1'][i] = nlp(train['question1'][i].decode('utf-8'))

0


/Users/cstahl/anaconda3/envs/python2.7/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10000
20000
30000


In [16]:
def make_vecs():
    vect_list = []
    results = []
    n = len(train)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = train.loc[i]
        q1, q2 = f['question1'], f['question2']
        results.append(f['is_duplicate'])
        try: 
            diff = parse(q1) - parse(q2)
            vect_list.append(diff)
        except Exception as e:
            print(i)
            print(e)
    return vect_list, results

vect_list, results = make_vecs()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [41]:
with open('word_vecs','w') as f:
    pickle.dump(vect_list,f)
with open('results','w') as f:
    pickle.dump(results,f)

In [43]:
vect_list_p = pickle.load(open('word_vecs', 'rb'))
results_p = pickle.load(open('results', 'rb'))

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer()
train = vectorizer.fit_transform(vect_list_p)
target = results_p

#### Validation

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf = MultinomialNB()
scores = cross_val_score(clf, train, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [4]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))

    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim
    
#unit test(s)
utest = train.loc[1]
print eval_sysim(utest)
utest = train.loc[3]
print eval_sysim(utest)
utest = train.loc[9]
print eval_sysim(utest)

0.934445572082
0.740894391357
0.819441244603


here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions


In [13]:
print train.loc[105780]

id                                      105780
qid1                                    174363
qid2                                    174364
question1       How can I develop android app?
question2                                  NaN
is_duplicate                                 0
Name: 105780, dtype: object
